# 🎙️ AI-Powered Client Registration via Voice

Welcome! This notebook uses AI to streamline client registration. You can speak a client's information, and the system will automatically transcribe it, extract the key details, and save them to a Google Sheet.

**Follow the steps below to get started.**

### Step 1: Install Necessary Libraries

Run the following cell to install the required Python packages for interacting with OpenAI and Google Sheets.

In [ ]:
!pip install -q openai gspread

### Step 2: Authenticate and Provide API Keys

Next, we need to connect to your Google Account and provide your OpenAI API key.

1.  **Google Authentication**: Running the first cell will prompt you to log in with your Google account. This is necessary to access Google Sheets.
2.  **OpenAI API Key**: The second cell will create a secure password field for you to enter your OpenAI API key.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
import openai
from getpass import getpass

OPENAI_API_KEY = getpass('Enter your OpenAI API key: ')
openai.api_key = OPENAI_API_KEY

### Step 3: Configure Your Google Sheet

1. Create a new Google Sheet.
2. Name it whatever you like (e.g., `Client Registrations`).
3. **Important**: In the Google Auth step above, you were assigned a client email. You must share your Google Sheet with that email address, giving it 'Editor' permissions. The email looks something like `...@...iam.gserviceaccount.com`.
4. Add the following headers to the first row of your sheet: `Name`, `Household Members`, `Address`, `Phone`.
5. Enter the exact name of your sheet in the cell below and run it.

In [ ]:
#@title Enter the name of your Google Sheet
GOOGLE_SHEET_NAME = 'Client Registrations' #@param {type:"string"}

try:
    worksheet = gc.open(GOOGLE_SHEET_NAME).sheet1
    print(f"Successfully connected to '{GOOGLE_SHEET_NAME}'!")
except gspread.exceptions.SpreadsheetNotFound:
    print(f"ERROR: Spreadsheet named '{GOOGLE_SHEET_NAME}' not found. Please check the name and sharing settings.")

### Step 4: Live Audio Recording Setup

This is the most complex part of the notebook. We use a combination of JavaScript and Python to record audio from your microphone.

- **How it works**: The `record_audio` function below creates a button. When you click it, it uses JavaScript in your browser to record audio, then sends that audio back to this notebook for processing.

Run this cell to define the helper functions.

In [ ]:
import json
from IPython.display import HTML, Audio, display
from google.colab.output import eval_js
from base64 import b64decode
import io
import textwrap

AUDIO_HTML = """
<script>
var my_div = document.createElement("div");
var my_p = document.createElement("p");
var my_btn = document.createElement("button");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_p);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 16000, //chrome seems to ignore, defaults to 48k
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... please wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()
sleep(2000).then(()=>{
  // wait 2000ms for the data to be available... 
  resolve(base64data.toString())

})
};
})
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  # Save to a file in memory
  audio_file = io.BytesIO(binary)
  audio_file.name = 'audio.webm'
  return audio_file

### Step 5: AI and Transcription Logic

Here are the functions that handle the AI part of our workflow:

- `transcribe_audio`: Takes the audio file we recorded and sends it to OpenAI's Whisper API to get a text transcript.
- `get_ai_response`: Manages the conversation with the GPT-4o model. It sends the conversation history and gets the AI's next question or the final JSON data.
- `SYSTEM_PROMPT`: This is the crucial instruction set for the AI, telling it how to act, what to ask for, and how to format the final output.

In [ ]:
def transcribe_audio(audio_file):
    """Sends audio to Whisper and returns the transcript."""
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
        return transcript.text
    except Exception as e:
        print(f"Error during transcription: {e}")
        return None

def get_ai_response(messages):
    """Sends messages to GPT-4o and gets a response."""
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            # response_format={"type": "json_object"} # This can be too restrictive for a conversation
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error getting AI response: {e}")
        return None

SYSTEM_PROMPT = """
You are an intelligent registration assistant for a non-profit organization.
Your goal is to collect the following information from a new client:
- Name
- Household Members (number of people)
- Full Address
- Phone Number

You must conduct a friendly, conversational interview. Ask one question at a time.
If the user's response is unclear, ask for clarification.

Once you are confident you have all four pieces of information, and only then,
your final response must be a single JSON object containing the extracted data.
The JSON object should have the following keys: "name", "household_members", "address", "phone".

Before you have all the information, your responses should be plain text questions.
When you are asking a question, your response must NOT be JSON. It must be a simple string.
For example: "Thanks! And how many people are in the household?"

Example of a final JSON response:
{
  "name": "Jane Doe",
  "household_members": 4,
  "address": "123 Main St, Anytown, USA 12345",
  "phone": "555-123-4567"
}

Start the conversation by introducing yourself and asking for the client's full name.
"""

### Step 6: Google Sheets Integration

This function takes the final extracted data and saves it as a new row in your Google Sheet.

In [ ]:
def save_to_sheet(client_data):
    """Formats and saves the client data to the Google Sheet."""
    try:
        # The order of headers in your Google Sheet
        header = ["Name", "Household Members", "Address", "Phone"]
        
        # Ensure all keys are present, defaulting to an empty string if not
        row_to_insert = [
            client_data.get("name", ""),
            client_data.get("household_members", ""),
            client_data.get("address", ""),
            client_data.get("phone", "")
        ]
        
        worksheet.append_row(row_to_insert)
        print("✅ Data successfully saved to Google Sheet!")
    except Exception as e:
        print(f"❌ Error saving to Google Sheet: {e}")

### Step 7: Run the Registration Process

All the setup is complete! Run the cell below to start the live, voice-driven registration process.

1. The AI will ask a question.
2. A button will appear. Press it to start recording your answer.
3. Press the button again to stop recording.
4. Your answer will be transcribed and shown to you.
5. The AI will ask the next question, and the process will repeat.
6. Once all information is gathered, you will be asked to review it and confirm before saving.

In [ ]:
# --- Main Application Flow ---

# Initialize conversation
messages = [{"role": "system", "content": SYSTEM_PROMPT}]
client_data = None

# Get the initial question from the AI
ai_response_text = get_ai_response(messages)
messages.append({"role": "assistant", "content": ai_response_text})

# Loop until we get JSON data
while True:
    # 1. Print AI question
    print(f"🤖 AI: {ai_response_text}")

    # 2. Record user's audio response
    audio_file = get_audio()

    # 3. Transcribe the audio
    print("\nTranscribing...")
    user_text = transcribe_audio(audio_file)
    if user_text:
        print(f"🎤 You said: {user_text}\n")

        # 4. Get AI response
        messages.append({"role": "user", "content": user_text})
        ai_response_text = get_ai_response(messages)

        # 5. Check if the response is the final JSON
        if ai_response_text.strip().startswith('{'):
            try:
                client_data = json.loads(ai_response_text)
                break # Exit the loop
            except json.JSONDecodeError:
                # The AI might have accidentally produced malformed JSON. Ask it to fix it.
                fix_request = "That doesn't look like valid JSON. Please provide the final data in the correct JSON format."
                messages.append({"role": "user", "content": fix_request})
                ai_response_text = get_ai_response(messages)
        else:
            messages.append({"role": "assistant", "content": ai_response_text})
    else:
        print("Could not transcribe audio, please try again.")

# --- Review and Save ---
print("\n" + "="*50)
print("✅ Registration Complete! Please review the extracted data:")
print("="*50 + "\n")

print(json.dumps(client_data, indent=2))

# Ask for confirmation to save
save_confirmation = input("\n\nDo you want to save this information to Google Sheets? (yes/no): ")
if save_confirmation.lower() == 'yes':
    save_to_sheet(client_data)
else:
    print("\nData not saved. Run this cell again to start over.")